# 自作簡易IPを用いたLED回路の動作確認ノートブック

myip_0というIPを自作してLED回路を組みました。
myip_0はデフォルトの「４つのレジスタ(32bit*4=128bit=16byte)の読み書きIP」において、内部レジスタslv_reg0[3:0]の内容を出力するよう書き換えたものです。たった２行程度の追加で、LEDの点滅がslv_reg0[3:0]と対応します。「FPGA大全」6-1に従って作成しました。
なお、このIPは標準IPの"AXI GPIO"におけるOutput GPIO Widthを4にしたものとほぼ同じものです。

Overlay("./led_overlay.bit").download()で回路(bitファイル)を読み込むことが出来る。

In [1]:
from pynq import Overlay
# load Base Overlay
Overlay("./myip.bit").download()

myip.bitは次のようになっている。
![](0607A_MYIP - [C__Users_ryuta_Documents_vivado_2016.1_0607A_MYIP_0607A_MYIP.xpr] - Vivado 2016.1 2017_06_15 20_30_49.png)

上記回路のmyip_0内にあるレジスタにアクセスすることでLEDの挙動を変化させることが出来る。tclファイルではSEG_myip_0_S00_AXI_regに対応する。myip_0のVerilogファイル名がmyip_0_S00_AXIであり、ビットストリームの生成時に回路の名前の先頭にSEG_が追加され、語尾に_Regが追加されるようだ。  

このSEG_myip_0_S00_AXI_regにアクセスするためには、そのベースアドレスを知る必要がある。PYNQではbitファイルの読み込みとともに、tclファイルの情報を解析し、ip_dictという名前の辞書が作成される。回路名とそのアドレス、オフセット、状態(デフォルトはNone)が一つにまとまっており、回路の名前からアドレスやその他の情報を調べることが出来る。

##### ip_dictについて(PYNQ公式ドキュメントより引用)
>The Overlay package generates a dictionary called ip_dict containing the names of IP in a specific overlay (e.g. base.bit). The dictionary can be used to reference an IP by name in your Python code, rather than by a hard coded address. It can also check the IP available in an overlay.  
...  
>Note, this parses the tcl file that was exported with the bitstream. It does not do check the overlay currently running in the PL.  
http://pynq.readthedocs.io/en/latest/16_creating_overlays.html

In [2]:
#ip_dictは辞書。ビットストリームファイルが読み込まれると同時に作成される。
from pynq import PL
PL.ip_dict

{'SEG_myip_0_S00_AXI_reg': ['0x43C00000', '0x00010000', None]}

### LEDを点灯させる(以下のコードではLEDが全て点灯する)

In [3]:
#"xilinx/pynq/mmio.py”と"xilinx/pynq/pl.py"を使っている
#MMIOクラスは/デバイスファイルdev/memを書き換えるためのもの。PLクラスはレジスタのアドレスを取得するためのもの。

from pynq import MMIO
from pynq import PL

led = MMIO(int(PL.ip_dict["SEG_myip_0_S00_AXI_reg"][0],16),4)
led.write(0x0,15) #第二引数が0xFでも0B1111でも同じ挙動を示す。

#### メモ
MMIO関数の説明
http://pynq.readthedocs.io/en/latest/pynq.html#module-pynq.mmio  
MMIOクラスのインスタンス作成時の第二引数はlengthだ。int型。説明を抜粋
> The length in bytes of the address range.  
アドレス範囲の長さ（バイト単位）

### 参考
- FPGA大全　6-1簡易IP作成機能を使った自作IPの作成  
https://www.amazon.co.jp/FPGA%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%9F%E3%83%B3%E3%82%B0%E5%A4%A7%E5%85%A8-Xilinx%E7%B7%A8-%E5%B0%8F%E6%9E%97-%E5%84%AA/dp/4798047538
- Document Creating Overlays  
http://pynq.readthedocs.io/en/latest/16_creating_overlays.html?highlight=tcl  
- Overlayとpythonのやり取りについて網羅しているサイト。  
http://todotani.cocolog-nifty.com/blog/2017/01/pynq-z1overlayp.html  
- xilinx/pynq/pl.pyのコード  
http://pynq.readthedocs.io/en/latest/_modules/pynq/pl.html  
- xilinx/pynq/mmio.pyのコード   
http://pynq.readthedocs.io/en/latest/_modules/pynq/mmio.html
- その他クラスのドキュメント    
http://pynq.readthedocs.io/en/latest/modules.html